<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/cifar10/Download_CIFAR10_models_from_zenodo_and_make_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import keras
from keras.models import load_model
from keras.models import Sequential, Model
print("keras version:", keras.__version__)
import tensorflow as tf
print("tensorflow version:", tf.__version__)
import random
import os
import sys
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras import backend as K
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


keras version: 2.2.4
tensorflow version: 1.14.0


In [2]:
batch_size = 128
num_classes = 10
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

full_x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
full_x_train /= 255
x_test /= 255
x_valid = full_x_train[-10000:]
print('x_train shape:', full_x_train.shape)
print(full_x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'valid samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
full_y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = full_y_train[-10000:]
y_test = keras.utils.to_categorical(y_test, num_classes)

output_file = "test_labels.txt"
f = open(output_file, 'w')
f.write("\n".join(["\t".join([str(x) for x in y]) for y in y_test]))
f.close()
os.system("gzip -f "+output_file)

output_file = "valid_labels.txt"
f = open(output_file, 'w')
f.write("\n".join(["\t".join([str(x) for x in y]) for y in y_valid]))
f.close()
os.system("gzip -f "+output_file)

output_file = "train_labels.txt"
f = open(output_file, 'w')
f.write("\n".join(["\t".join([str(x) for x in y]) for y in full_y_train]))
f.close()
os.system("gzip -f "+output_file)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 valid samples
10000 test samples


0

In [3]:
model_files = []
for seed in range(0,100,10):
    np.random.seed(seed)
    random.seed(seed)
    for model_idx,train_set_size in enumerate([250, 500, 1000, 2000, 4000,
                                           8000, 16000]):
        model_file = "model_mnist_set-"+str(train_set_size)+"_seed-"+str(seed)+".h5"
        model_files.append(model_file)
        print("On train set size",train_set_size)

        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes))
        model.add(Activation("softmax"))

        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adadelta(),
                      metrics=['accuracy'])
        x_train = full_x_train[:train_set_size] 
        y_train = full_y_train[:train_set_size]
        print("Mean y train:",np.mean(y_train, axis=0))
        print("Mean y valid:",np.mean(y_valid, axis=0))
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x_valid, y_valid),
                  callbacks=[EarlyStopping(
                    monitor='val_loss', patience=5,
                    restore_best_weights=True)])
        model.save(model_file)

        pre_softmax_model = Model(input=model.input,
                            output=model.layers[-2].output)
        print("Making predictions on validation set")
        valid_preacts = pre_softmax_model.predict(x_valid)
        print("Making predictions on test set")
        test_preacts = pre_softmax_model.predict(x_test)
        print('Test accuracy:', np.mean(np.argmax(test_preacts,axis=-1)
                                        ==np.argmax(y_test,axis=-1)))
        print('Valid accuracy:', np.mean(np.argmax(valid_preacts,axis=-1)
                                        ==np.argmax(y_valid,axis=-1)))
        sys.stdout.flush()
        test_predictions_file = ("testpreacts_"+model_file.split(".")[0])+".txt"
        print("Saving", test_predictions_file)
        f = open(test_predictions_file,'w')
        for test_preact in test_preacts:
            f.write("\t".join([str(x) for x in test_preact])+"\n") 
        f.close()
        !md5sum $test_predictions_file
        !gzip $test_predictions_file

        valid_predictions_file = ("validpreacts_"+model_file.split(".")[0])+".txt"
        print("Saving", valid_predictions_file)
        f = open(valid_predictions_file,'w')
        for valid_preact in valid_preacts:
            f.write("\t".join([str(x) for x in valid_preact])+"\n") 
        f.close()
        !md5sum $valid_predictions_file
        !gzip $valid_predictions_file

On train set size 250





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Mean y train: [0.12  0.136 0.092 0.108 0.096 0.068 0.096 0.104 0.076 0.104]
Mean y valid: [0.0991 0.1064 0.099  0.103  0.0983 0.0915 0.0967 0.109  0.1009 0.0961]
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 250 samples, validate on 10000 samples
Epoch 1/50
250/250 [==============================] - 3s 11ms/step - loss: 2.2851 - acc: 0.1720 - val_loss: 2.1740 - val_acc: 0.3403
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 2.0759 - acc: 0.3920 - val_loss: 1.9153 - val_acc: 0.4298
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 1.7751 - acc: 0.4400 - val_loss: 1.6914 - val_acc: 0.5132
Epoch 4/50
250/250 [==============================] - 1s 2ms/step - loss: 1.6777 - acc: 0.4160 - val_loss: 1.5967 - val_acc: 0.4975
Epoch 5/50
250/250 [===

/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/ipykernel/__main__.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on test set
Test accuracy: 0.822
Valid accuracy: 0.8306
Saving testpreacts_model_mnist_set-250_seed-0.txt
dfe6891cf3ece30bb467e9e27ac06f7b  testpreacts_model_mnist_set-250_seed-0.txt
Saving validpreacts_model_mnist_set-250_seed-0.txt
a7658d4b6014ce714322f9309119052e  validpreacts_model_mnist_set-250_seed-0.txt
On train set size 500
Mean y train: [0.1   0.132 0.104 0.1   0.104 0.078 0.09  0.104 0.078 0.11 ]
Mean y valid: [0.0991 0.1064 0.099  0.103  0.0983 0.0915 0.0967 0.109  0.1009 0.0961]
Train on 500 samples, validate on 10000 samples
Epoch 1/50
500/500 [==============================] - 1s 3ms/step - loss: 2.1826 - acc: 0.3140 - val_loss: 1.9309 - val_acc: 0.4914
Epoch 2/50
500/500 [==============================] - 1s 1ms/step - loss: 1.6237 - acc: 0.5060 - val_loss: 1.3439 - val_acc: 0.5192
Epoch 3/50
500/500 [==============================] - 1s 1ms/step - loss: 1.3164 - acc: 0.5920 - val_loss: 0.8789 - val_acc: 0.7086
Epoch 4/50
500/500 [=====================

/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/ipykernel/__main__.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on validation set
Making predictions on test set
Test accuracy: 0.8867
Valid accuracy: 0.8949
Saving testpreacts_model_mnist_set-500_seed-0.txt
b9079d61b2b1d0a2ce52bc366d8d8b33  testpreacts_model_mnist_set-500_seed-0.txt
Saving validpreacts_model_mnist_set-500_seed-0.txt
79656900f0c194df8e449dad6319ffe6  validpreacts_model_mnist_set-500_seed-0.txt
On train set size 1000
Mean y train: [0.097 0.116 0.099 0.093 0.105 0.092 0.094 0.117 0.087 0.1  ]
Mean y valid: [0.0991 0.1064 0.099  0.103  0.0983 0.0915 0.0967 0.109  0.1009 0.0961]
Train on 1000 samples, validate on 10000 samples
Epoch 1/50
1000/1000 [==============================] - 2s 2ms/step - loss: 1.8908 - acc: 0.3450 - val_loss: 1.3567 - val_acc: 0.5270
Epoch 2/50
1000/1000 [==============================] - 1s 802us/step - loss: 1.2590 - acc: 0.5820 - val_loss: 0.7985 - val_acc: 0.7661
Epoch 3/50
1000/1000 [==============================] - 1s 743us/step - loss: 0.8144 - acc: 0.7420 - val_loss: 0.5692 - val_acc

...

Making predictions on validation set
Making predictions on test set
Test accuracy: 0.9693
Valid accuracy: 0.9718
Saving testpreacts_model_mnist_set-4000_seed-90.txt
4cafd7224115c3a6715fa4be4f0acda7  testpreacts_model_mnist_set-4000_seed-90.txt
Saving validpreacts_model_mnist_set-4000_seed-90.txt
e447566d5771849151e38148886fde97  validpreacts_model_mnist_set-4000_seed-90.txt
On train set size 8000
Mean y train: [0.099375 0.11325  0.0985   0.101375 0.099875 0.08775  0.09975  0.106625
 0.094125 0.099375]
Mean y valid: [0.0991 0.1064 0.099  0.103  0.0983 0.0915 0.0967 0.109  0.1009 0.0961]
Train on 8000 samples, validate on 10000 samples
Epoch 1/50
8000/8000 [==============================] - 10s 1ms/step - loss: 0.8149 - acc: 0.7388 - val_loss: 0.2589 - val_acc: 0.9290
Epoch 2/50
8000/8000 [==============================] - 3s 407us/step - loss: 0.2682 - acc: 0.9229 - val_loss: 0.1983 - val_acc: 0.9375
Epoch 3/50
8000/8000 [==============================] - 3s 406us/step - loss: 0.1822 - 

/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/ipykernel/__main__.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on validation set
Making predictions on test set
Test accuracy: 0.9789
Valid accuracy: 0.9801
Saving testpreacts_model_mnist_set-8000_seed-90.txt
b9dc4aada56c19b9abc88a5d3365fc0b  testpreacts_model_mnist_set-8000_seed-90.txt
Saving validpreacts_model_mnist_set-8000_seed-90.txt
a32113abf5f485a86cd8bba4b0540ed2  validpreacts_model_mnist_set-8000_seed-90.txt
On train set size 16000
Mean y train: [0.100375  0.113     0.0971875 0.1039375 0.097     0.0885    0.099375
 0.10575   0.0950625 0.0998125]
Mean y valid: [0.0991 0.1064 0.099  0.103  0.0983 0.0915 0.0967 0.109  0.1009 0.0961]
Train on 16000 samples, validate on 10000 samples
Epoch 1/50
16000/16000 [==============================] - 13s 817us/step - loss: 0.6065 - acc: 0.8106 - val_loss: 0.1706 - val_acc: 0.9507
Epoch 2/50
16000/16000 [==============================] - 6s 372us/step - loss: 0.1865 - acc: 0.9437 - val_loss: 0.0888 - val_acc: 0.9760
Epoch 3/50
16000/16000 [==============================] - 6s 365us/ste

/users/amr1/miniconda3/envs/basepair/lib/python3.6/site-packages/ipykernel/__main__.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Making predictions on validation set
Making predictions on test set
Test accuracy: 0.9847
Valid accuracy: 0.9843
Saving testpreacts_model_mnist_set-16000_seed-90.txt
08500b34a2bf3a2f7ea58d6e75825fdd  testpreacts_model_mnist_set-16000_seed-90.txt
Saving validpreacts_model_mnist_set-16000_seed-90.txt
14945a35ca722877d1f67326b5e27640  validpreacts_model_mnist_set-16000_seed-90.txt
